### 공고 데이터 전처리

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re

In [4]:
# load the dataset
gongo = pd.read_csv('..\\gongodataset\\gongo_raw.csv')

#### 필요없는 열 삭제

In [6]:
# drop 직무(복수선택) column cause it's same as the 직무_lv2 column
gongo.drop(['직무(복수선택)'],axis=1, inplace=True)

#### 데이터 타입 변환

In [7]:
# '스킬' column:  str -> list
for i in range(len(gongo)):
  gongo['스킬'][i] = gongo['스킬'][i].strip("[").strip("]").replace("'","").split(", ")

In [8]:
# '직무_lv1' column:  str -> list
for i in range(len(gongo)):
  gongo['직무_lv1'][i] = gongo['직무_lv1'][i].strip("[").strip("]").replace("'","").split(", ")

In [9]:
# '직무_lv2' column:  str -> list
for i in range(len(gongo)):
  gongo['직무_lv2'][i] = gongo['직무_lv2'][i].strip("[").strip("]").replace("'","").split(", ")

#### 기업 소개 ~ 우대사항 컬럼의 nan값 채우기

In [10]:
# check
gongo[:3]

,회사명,고용 형태,스킬,기업 소개,주요 업무,자격 요건,우대 사항,채용 절차,복리후생,회사위치,직무_lv1,직무_lv2
0,캐롯손해보험(주),정규직,"[Oracle, BI, SQL, dw, etl]","캐롯손해보험은 한화, SKT, 현대자동차, 알토스벤쳐스, 스틱인베스트먼트 등 대형 ...",• 캐롯 주행/계약 관련 지표 생성 및 제공해요.\r\n• Data-warehous...,• BI 및 ETL 관련 분야 경력 5년 이상이신 분이 필요해요.\r\n• DW 데...,• BI 툴 활용 및 Hadoop에 대한 연동을 이해하신 분이면 좋아요.\r\n• ...,"• 해당 직무는 수시채용으로, 채용 완료 시 공고가 마감될 수 있어요.\r\n• 지...",[성장을 위한 휴식]\r\n• 매년 복지포인트 200만원\r\n• 설 차례비 지급\...,"서울 중구 을지로100 파인에비뉴 B동 20층, 캐롯손해보험",[데이터],[BI 엔지니어]
1,(주)웅진,정규직,[POWER BI],NaN,"- MS Power Platform 프로젝트 수행 및 기술지원 (Automate, ...",【지원자격】 \r\n - 모집직급: 대리~과장 \r\n - MS Power Plat...,NaN,NaN,NaN,서울 중구 청계천로 24,"[개발, 데이터]","[클라우드 개발, BI 엔지니어]"
2,잡플래닛,정규직,"[데이터분석, BI, python, SQL, Tableau, POWER BI]",더 나은 선택은 없을까?\r\n 지금 이 순간에도 잡플래닛을 방문하는 월 300만명...,[조직소개]\r\n잡플래닛의 데이터 조직은 Data 1st전략으로 서비스의 핵심 자...,"[자격 조건]\r\n• SQL 상급, 파이썬 중급\r\n• Data Visualiz...","[우대 사항]\r\n• Statistics, Econometrics, Applied...",[전형 프로세스]\r\n• 서류전형 -> 1차 실무 인터뷰 -> 2차 협업부서 인터...,- 분 단위 시차 출퇴근제 (오전 7시~ 오전 10시 사이 자율 출퇴근)\r\n ...,"서울특별시 강남구 테헤란로 509 엔씨타워 I, 10층",[데이터],[BI 엔지니어]


In [11]:
# # of nan by columns
for column_name in gongo.columns:
  print(column_name, gongo[column_name].isna().sum())

회사명 0
고용 형태 0
스킬 0
기업 소개 537
주요 업무 0
자격 요건 0
우대 사항 413
채용 절차 1930
복리후생 455
회사위치 36
직무_lv1 0
직무_lv2 0


In [12]:
# combine '자격 요건' and '우대 사항' column to make '자격요건 및 우대사항' column
gongo['자격요건 및 우대사항'] = gongo['자격 요건'] + gongo['우대 사항']

In [13]:
# dictionary
position = '담당업무|담당 업무|주요업무|주요 업무|직무설명|직무 설명|어떤 일을 하나요?|직무소개|직무 소개' # 주요 업무
qualification = '지원자격|지원 자격|필수조건|필수 조건|공통자격|공통 자격|자격요건|자격 요건|기술스택|기술 스택' # 자격 요건
prefer = '우대사항|우대사항' # 우대 사항

position_list = ['담당업무','담당 업무','주요업무','주요 업무','직무설명','직무 설명','어떤 일을 하나요?','직무소개','직무 소개','포지션 정보']
qualification_list = ['지원자격','지원 자격','필수조건','필수 조건','공통자격','공통 자격','자격요건','자격 요건','기술스택','기술 스택']
prefer_list = ['우대사항','우대 사항']

# company = '회사설명|회사 설명|기업 소개|기업소개|팀 소개|팀소개|조직소개|조직 소개' # 기업 소개
# hiring_process = '합류|채용절차|채용 절차' # 채용절차

In [14]:
# 주요 업무 column
for i in range(len(gongo)):
  # if a word from position dic is appeared: the content before that should be cut
  for pos_word in position_list:
    pos_word_index = gongo['주요 업무'][i].find(pos_word)
    if pos_word_index!=-1: # other contents are appeared
      gongo['주요 업무'][i] = gongo['주요 업무'][i][pos_word_index+len(pos_word):]

  # if a word from qualification dic is appeared: the content after that should be cut
  for qual_word in qualification_list:
    qual_word_index = gongo['주요 업무'][i].find(qual_word)
    if qual_word_index!=-1:
      gongo['주요 업무'][i] = gongo['주요 업무'][i][:qual_word_index]

  # if a word from prefer dic is appeared: the content after that should be cut
  for prefer_word in prefer_list:
    prefer_word_index = gongo['주요 업무'][i].find(prefer_word)
    if prefer_word_index!=-1: # 다른 내용이 있는 경우
      gongo['주요 업무'][i] = gongo['주요 업무'][i][:prefer_word_index]

In [15]:
# 자격요건 및 우대사항 column
# qual words are appeared: cut the before words
for i in range(len(gongo)):
  if pd.isna(gongo['자격요건 및 우대사항'][i]):
    continue
  else:
    for qual_word in qualification_list:
      qual_word_index = gongo['자격요건 및 우대사항'][i].find(qual_word)
      if qual_word_index!=-1:
        gongo['자격요건 및 우대사항'][i] = gongo['자격요건 및 우대사항'][i][qual_word_index+len(qual_word):]

In [16]:
# check
gongo[:3]

,회사명,고용 형태,스킬,기업 소개,주요 업무,자격 요건,우대 사항,채용 절차,복리후생,회사위치,직무_lv1,직무_lv2,자격요건 및 우대사항
0,캐롯손해보험(주),정규직,"[Oracle, BI, SQL, dw, etl]","캐롯손해보험은 한화, SKT, 현대자동차, 알토스벤쳐스, 스틱인베스트먼트 등 대형 ...",• 캐롯 주행/계약 관련 지표 생성 및 제공해요.\r\n• Data-warehous...,• BI 및 ETL 관련 분야 경력 5년 이상이신 분이 필요해요.\r\n• DW 데...,• BI 툴 활용 및 Hadoop에 대한 연동을 이해하신 분이면 좋아요.\r\n• ...,"• 해당 직무는 수시채용으로, 채용 완료 시 공고가 마감될 수 있어요.\r\n• 지...",[성장을 위한 휴식]\r\n• 매년 복지포인트 200만원\r\n• 설 차례비 지급\...,"서울 중구 을지로100 파인에비뉴 B동 20층, 캐롯손해보험",[데이터],[BI 엔지니어],• BI 및 ETL 관련 분야 경력 5년 이상이신 분이 필요해요.\r\n• DW 데...
1,(주)웅진,정규직,[POWER BI],NaN,"- MS Power Platform 프로젝트 수행 및 기술지원 (Automate, ...",【지원자격】 \r\n - 모집직급: 대리~과장 \r\n - MS Power Plat...,NaN,NaN,NaN,서울 중구 청계천로 24,"[개발, 데이터]","[클라우드 개발, BI 엔지니어]",NaN
2,잡플래닛,정규직,"[데이터분석, BI, python, SQL, Tableau, POWER BI]",더 나은 선택은 없을까?\r\n 지금 이 순간에도 잡플래닛을 방문하는 월 300만명...,"]\r\n• 전사 Business Intelligence : Dashboard, R...","[자격 조건]\r\n• SQL 상급, 파이썬 중급\r\n• Data Visualiz...","[우대 사항]\r\n• Statistics, Econometrics, Applied...",[전형 프로세스]\r\n• 서류전형 -> 1차 실무 인터뷰 -> 2차 협업부서 인터...,- 분 단위 시차 출퇴근제 (오전 7시~ 오전 10시 사이 자율 출퇴근)\r\n ...,"서울특별시 강남구 테헤란로 509 엔씨타워 I, 10층",[데이터],[BI 엔지니어],"[자격 조건]\r\n• SQL 상급, 파이썬 중급\r\n• Data Visualiz..."


#### 인덱스 컬럼 부여

In [17]:
# add index column
gongo["index"] = gongo.index

#### 문자열 정제

In [18]:
# regular expression
for i, string in enumerate(gongo['주요 업무']):
    if type(string)!=str:
        continue
    else:
        gongo['주요 업무'][i] = re.compile(r'[^\w\s]').sub('',string)

for i, string in enumerate(gongo['자격요건 및 우대사항']):
    if type(string)!=str:
        continue
    else:
        gongo['자격요건 및 우대사항'][i] = re.compile(r'[^\w\s]').sub('',string)

C:\Users\Andlab\AppData\Local\Temp\ipykernel_7108\3200414828.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gongo['주요 업무'][i] = re.compile(r'[^\w\s]').sub('',string)
C:\Users\Andlab\AppData\Local\Temp\ipykernel_7108\3200414828.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gongo['자격요건 및 우대사항'][i] = re.compile(r'[^\w\s]').sub('',string)


In [20]:
# remove 'ㆍ'
gongo["주요 업무"] = gongo["주요 업무"].str.replace(pat=r'[ㆍ]', repl=r'', regex=True)

gongo["자격요건 및 우대사항"] = gongo["자격요건 및 우대사항"].str.replace(pat=r'[ㆍ]', repl=r'', regex=True)

# '\n'제거
gongo["주요 업무"] = gongo["주요 업무"].str.replace(pat=r'[\n\t\r]', repl=r'', regex=True)

gongo["자격요건 및 우대사항"] = gongo["자격요건 및 우대사항"].str.replace(pat=r'[\n\t\r]', repl=r'', regex=True)

In [22]:
# drop NaN
gongo.dropna(subset = ['주요 업무','자격요건 및 우대사항'], inplace=True)
gongo[:3]

,회사명,고용 형태,스킬,기업 소개,주요 업무,자격 요건,우대 사항,채용 절차,복리후생,회사위치,직무_lv1,직무_lv2,자격요건 및 우대사항,index
0,캐롯손해보험(주),정규직,"[Oracle, BI, SQL, dw, etl]","캐롯손해보험은 한화, SKT, 현대자동차, 알토스벤쳐스, 스틱인베스트먼트 등 대형 ...",캐롯 주행계약 관련 지표 생성 및 제공해요 Datawarehouse 및 Data ...,• BI 및 ETL 관련 분야 경력 5년 이상이신 분이 필요해요.\r\n• DW 데...,• BI 툴 활용 및 Hadoop에 대한 연동을 이해하신 분이면 좋아요.\r\n• ...,"• 해당 직무는 수시채용으로, 채용 완료 시 공고가 마감될 수 있어요.\r\n• 지...",[성장을 위한 휴식]\r\n• 매년 복지포인트 200만원\r\n• 설 차례비 지급\...,"서울 중구 을지로100 파인에비뉴 B동 20층, 캐롯손해보험",[데이터],[BI 엔지니어],BI 및 ETL 관련 분야 경력 5년 이상이신 분이 필요해요 DW 데이터 모델링 ...,0
2,잡플래닛,정규직,"[데이터분석, BI, python, SQL, Tableau, POWER BI]",더 나은 선택은 없을까?\r\n 지금 이 순간에도 잡플래닛을 방문하는 월 300만명...,전사 Business Intelligence Dashboard Report 개발...,"[자격 조건]\r\n• SQL 상급, 파이썬 중급\r\n• Data Visualiz...","[우대 사항]\r\n• Statistics, Econometrics, Applied...",[전형 프로세스]\r\n• 서류전형 -> 1차 실무 인터뷰 -> 2차 협업부서 인터...,- 분 단위 시차 출퇴근제 (오전 7시~ 오전 10시 사이 자율 출퇴근)\r\n ...,"서울특별시 강남구 테헤란로 509 엔씨타워 I, 10층",[데이터],[BI 엔지니어],자격 조건 SQL 상급 파이썬 중급 Data Visualization 소프트웨어Ta...,2
3,(주)포바이포,정규직,"[재무, 세무조사, 손익분석, 재무총괄]",“Into the NEW ERA” \r\n\r\n2017년에 설립된 주식회사 포바이...,재무총괄회계세무자금 연간 경영계획 수립 장기적 재무적 손익분석 및 예측 MA 업무...,• 기업재무 총괄 책임자 경험 1년 이상 \r\n• 해당 직무 경력 10년 이상\r...,• 빠르게 성장하는 IT / 스타트업을 경험해보신 분\r\n• 1년 이상의 CFO ...,NaN,[근무환경]\r\n- 월요일 출근 1pm - 퇴근 7pm (화~금 10am-7pm...,"서울 서초구 강남대로 479, 12층,13층,14층",[기획/경영],[CEO (Chief Executive Officer)],기업재무 총괄 책임자 경험 1년 이상 해당 직무 경력 10년 이상 회계법인 경험...,3


In [23]:
# save as csv
gongo.to_csv("..\\gongodataset\\gongo_preprocessed.csv", index=False, encoding='utf-8')